In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', None)

from sklearn import *
from lightgbm import LGBMRegressor

/Users/amane/.pyenv/versions/3.6.1/envs/compe/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/amane/.pyenv/versions/3.6.1/envs/compe/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/amane/.pyenv/versions/3.6.1/envs/compe/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into w

In [15]:
log1p_beforehand = True

In [28]:
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
from xgboost import XGBRegressor

data = {
    'tra': pd.read_csv('../input/air_visit_data.csv'),
    'as': pd.read_csv('../input/air_store_info.csv'),
    'hs': pd.read_csv('../input/hpg_store_info.csv'),
    'ar': pd.read_csv('../input/air_reserve.csv'),
    'hr': pd.read_csv('../input/hpg_reserve.csv'),
    'id': pd.read_csv('../input/store_id_relation.csv'),
    'tes': pd.read_csv('../input/sample_submission.csv'),
    'hol': pd.read_csv('../input/date_info.csv').rename(columns={'calendar_date':'visit_date'})
    }

if log1p_beforehand:
    data['tra']['visitors'] = np.log1p(data['tra']['visitors'])

data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])

for df in ['ar','hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    tmp1 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
    tmp2 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
    data[df] = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])

data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

unique_stores = data['tes']['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)

#sure it can be compressed...
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

stores = pd.merge(stores, data['as'], how='left', on=['air_store_id']) 
# NEW FEATURES FROM Georgii Vyshnia
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))
lbl = preprocessing.LabelEncoder()
for i in range(10):
    stores['air_genre_name'+str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
    stores['air_area_name'+str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date']) 

train = pd.merge(train, stores, how='left', on=['air_store_id','dow']) 
test = pd.merge(test, stores, how='left', on=['air_store_id','dow'])

for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

train['total_reserv_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserv_mean'] = (train['rv2_x'] + train['rv2_y']) / 2
train['total_reserv_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y']) / 2

test['total_reserv_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserv_mean'] = (test['rv2_x'] + test['rv2_y']) / 2
test['total_reserv_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y']) / 2

# NEW FEATURES FROM JMBULL
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

# NEW FEATURES FROM Georgii Vyshnia
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2'] = lbl.transform(test['air_store_id'])

train = train.fillna(-1)
test = test.fillna(-1)

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [29]:
test.shape

(32019, 54)

In [30]:
col = [c for c in train if c not in ['id', 'air_store_id', 'air_area_name', 'air_genre_name', 'visit_date','visitors']]

In [34]:
train.groupby('air_store_id').visitors.mean()

air_store_id
air_00a91d42b08b08d9    3.167414
air_0164b9927d20bcc3    2.117522
air_0241aa3964b7f861    2.236649
air_0328696196e46f18    1.966041
air_034a3d5b40d5b1b1    2.457141
air_036d4f1ee7285390    3.016290
air_0382c794b73b51ad    3.014849
air_03963426c9312048    3.466131
air_04341b588bde96cd    3.488073
air_049f6d5b402a31b2    2.367463
air_04cae7c1bc9b2a0b    2.970835
air_0585011fa179bcce    1.654938
air_05c325d315cc17f5    3.067861
air_0647f17b4dc041c8    3.396164
air_064e203265ee5753    3.037076
air_066f0221b8a4d533    2.472980
air_06f95ac5c33aca10    2.973037
air_0728814bd98f7367    2.183842
air_0768ab3910f7967f    2.547386
air_07b314d83059c4d2    3.436534
air_07bb665f9cdfbdfb    3.279341
air_082908692355165e    3.487676
air_083ddc520ea47e1e    2.209605
air_0845d8395f30c6bb    2.926955
air_084d98859256acf0    2.637420
air_0867f7bebad6a649    2.917509
air_08ba8cd01b3ba010    2.301373
air_08cb3c4ee6cd6a22    2.605300
air_08ef81d5b7a0d13f    2.531565
air_08f994758a1e76d4    3.1222

In [35]:
def target_encode(col):
    global train, test
    ref = train.loc[~val_id].groupby(col)['visitors'].mean().to_frame(name=col+'_target')
    train = pd.merge(left=train, right=ref, left_on=col, right_index=True, how='left')
    ref = train.groupby(col)['visitors'].mean().to_frame(name=col+'_target')
    test = pd.merge(left=test, right=ref, left_on=col, right_index=True, how='left')

In [36]:
val_id = ((train.year == 2017) & (train.month >= 3))
print('val ratio:', f'{len(train.loc[val_id])/len(train):.2%}')

val ratio: 15.10%


In [37]:
target_encode('air_store_id')
target_encode('air_genre_name')
target_encode('air_area_name')

In [47]:
col = [c for c in train if c not in ['id', 'air_store_id', 'air_area_name', 'air_genre_name', 'visit_date','visitors']]
if not log1p_beforehand:
    train['visitors'] = np.log1p(train['visitors'])
X_trn = train.loc[~val_id, col]
y_trn = train.loc[~val_id, 'visitors'].values
X_val = train.loc[val_id, col]
y_val = train.loc[val_id, 'visitors'].values
X_all = train[col]
y_all = train['visitors'].values
X_tst = test[col]

In [48]:
# params = {
#     'boosting_type': 'gbdt',
#     'num_leaves': 70,
#     'n_estimators': 500,
#     'learning_rate': 0.05,
#     'random_state': 77,
#     'max_depth': 6,
#     'min_child_samples': 50,
#     'colsample_bytree': 0.8,
#     'subsample': 0.8
# }
params = {'colsample_bytree': 0.8, 'learning_rate': 0.05, 
 'min_child_samples': 50, 'n_estimators': 600, 'num_leaves': 70, 'random_state': 77, 'subsample': 1.0}
lgb = LGBMRegressor(**params)

In [49]:
lgb.fit(X_trn, y_trn)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
       learning_rate=0.05, max_depth=-1, min_child_samples=50,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=600,
       n_jobs=-1, num_leaves=70, objective=None, random_state=77,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1)

In [50]:
print(f'train: {RMSLE(y_trn, lgb.predict(X_trn)):.4f}, val: {RMSLE(y_val, lgb.predict(X_val)):.4f}')
print('params = ', params)
print('cols = ', list(X_trn.columns))

train: 0.4581, val: 0.4855
params =  {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'min_child_samples': 50, 'n_estimators': 600, 'num_leaves': 70, 'random_state': 77, 'subsample': 1.0}
cols =  ['dow', 'year', 'month', 'day_of_week', 'holiday_flg', 'min_visitors', 'mean_visitors', 'median_visitors', 'max_visitors', 'count_observations', 'latitude', 'longitude', 'air_genre_name0', 'air_area_name0', 'air_genre_name1', 'air_area_name1', 'air_genre_name2', 'air_area_name2', 'air_genre_name3', 'air_area_name3', 'air_genre_name4', 'air_area_name4', 'air_genre_name5', 'air_area_name5', 'air_genre_name6', 'air_area_name6', 'air_genre_name7', 'air_area_name7', 'air_genre_name8', 'air_area_name8', 'air_genre_name9', 'air_area_name9', 'rs1_x', 'rv1_x', 'rs2_x', 'rv2_x', 'rs1_y', 'rv1_y', 'rs2_y', 'rv2_y', 'total_reserv_sum', 'total_reserv_mean', 'total_reserv_dt_diff_mean', 'date_int', 'var_max_lat', 'var_max_long', 'lon_plus_lat', 'air_store_id2', 'air_store_id_target', 'air_genre_name_target'

In [51]:
lgb.fit(X_all, y_all)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
       learning_rate=0.05, max_depth=-1, min_child_samples=50,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=600,
       n_jobs=-1, num_leaves=70, objective=None, random_state=77,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1)

In [52]:
test['visitors'] = np.expm1(lgb.predict(X_tst))
filename = '../output/180130_lgb7'
test[['id', 'visitors']].to_csv(filename+'.csv.gz', index=False, compression='gzip')
test[['id', 'visitors']].to_csv(filename+'.csv', index=False)

In [53]:
test.shape

(32019, 57)

In [56]:
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')